In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import os
import json
import matplotlib.pyplot as plt

In [10]:
root_dir = '/pfs/work7/workspace/scratch/ma_xinygao-team_project_fss2024/benchmarking_robustness/optical_flow_estimation/outputs/validate_weather'
results_dir = '/pfs/work7/workspace/scratch/ma_xinygao-team_project_fss2024/benchmarking_robustness/optical_flow_estimation/outputs/tables'
plot_dir = '/pfs/work7/workspace/scratch/ma_xinygao-team_project_fss2024/benchmarking_robustness/optical_flow_estimation/outputs/graphs'
# Used to store various weather types
dfs = {}

# Traverse the subfolders in the root directory (such as fog, rain, snow, etc.)
for weather in os.listdir(root_dir):
    weather_dir = os.path.join(root_dir, weather)
    
    # Make sure the path is a directory
    if os.path.isdir(weather_dir):
        # Lists storing data
        data = []

        # Traverse the subfolders in the weather folder (such as craft, flowformer, etc.)
        for model in os.listdir(weather_dir):
            model_dir = os.path.join(weather_dir, model)
            
            # Make sure the path is a directory
            if os.path.isdir(model_dir):
                metric_path = os.path.join(model_dir, 'metrics_sintel-tr115_final.json')
                
                # Reading json files
                if os.path.exists(metric_path):
                    with open(metric_path, 'r') as f:
                        metric_data = json.load(f)
                        
                        # Get the experiments list
                        experiments = metric_data.get('experiments', [])
                        
                        # 初Initialize the dictionary to store the epe value
                        epe_values = {
                            'targeted_true_zero': None,
                            'targeted_false_zero': None,
                            'targeted_true_negative': None,
                            'targeted_false_negative': None
                        }
                        
                        # Traverse experiments and find the target field
                        for experiment in experiments:
                            targeted = experiment.get('targeted')
                            target = experiment.get('target')
                            
                            if targeted == True and target == 'zero':
                                epe_values['targeted_true_zero'] = experiment.get('metrics', {}).get('epe')
                            elif targeted == False and target == 'zero':
                                epe_values['targeted_false_zero'] = experiment.get('metrics', {}).get('epe')
                            elif targeted == True and target == 'negative':
                                epe_values['targeted_true_negative'] = experiment.get('metrics', {}).get('epe')
                            elif targeted == False and target == 'negative':
                                epe_values['targeted_false_negative'] = experiment.get('metrics', {}).get('epe')
                        
                        # Extract model name
                        model_name = model.split('_')[0]
                        
                        # Add the results to the data list
                        data.append([
                            model_name, 
                            epe_values['targeted_true_zero'], 
                            epe_values['targeted_false_zero'], 
                            epe_values['targeted_true_negative'], 
                            epe_values['targeted_false_negative']
                        ])

        # Creating a DataFrame
        df = pd.DataFrame(data, columns=[
            'model', 
            'epe_targeted_true_zero', 
            'epe_targeted_false_zero', 
            'epe_targeted_true_negative', 
            'epe_targeted_false_negative'
        ])
        
        # Save DataFrame to Dictionary
        dfs[weather] = df

        csv_path = os.path.join(results_dir, f'{weather}_model_epe.csv')
        df.to_csv(csv_path, index=False)
        print(f"Saved DataFrame for {weather} to {csv_path}")

# Print the name and contents of each DataFrame
for weather, df in dfs.items():
    print(f"DataFrame for {weather}:\n", df, "\n")

    # Plotting
    for column in ['epe_targeted_true_zero', 
                       'epe_targeted_false_zero', 
                       'epe_targeted_true_negative', 
                       'epe_targeted_false_negative']:
            
            plt.figure(figsize=(10, 6))
            plt.bar(df['model'], df[column], color='skyblue')
            plt.title(f'{column} for {weather}')
            plt.xlabel('Model')
            plt.ylabel(column)
            plt.xticks(rotation=45)
            plt.tight_layout()
            
            # Save the plot
            plot_path = os.path.join(plot_dir, f'weather_{weather}_{column}.png')
            plt.savefig(plot_path)
            print(f"Saved plot for {column} in {weather} to {plot_path}")
            plt.close()

# 打印每个天气情况的 DataFrame 的名称和内容
for weather, df in dfs.items():
    print(f"DataFrame for {weather}:\n", df, "\n")



Saved DataFrame for rain to /pfs/work7/workspace/scratch/ma_xinygao-team_project_fss2024/benchmarking_robustness/optical_flow_estimation/outputs/tables/rain_model_epe.csv
Saved DataFrame for snow to /pfs/work7/workspace/scratch/ma_xinygao-team_project_fss2024/benchmarking_robustness/optical_flow_estimation/outputs/tables/snow_model_epe.csv
Saved DataFrame for sparks to /pfs/work7/workspace/scratch/ma_xinygao-team_project_fss2024/benchmarking_robustness/optical_flow_estimation/outputs/tables/sparks_model_epe.csv
Saved DataFrame for fog to /pfs/work7/workspace/scratch/ma_xinygao-team_project_fss2024/benchmarking_robustness/optical_flow_estimation/outputs/tables/fog_model_epe.csv
DataFrame for rain:
           model  epe_targeted_true_zero  epe_targeted_false_zero  \
0      ms-raft+               13.258510                 4.748733   
1           gma               14.264215                 4.732660   
2  flowformer++               13.267375                 3.474418   
3          raft      